In [ ]:
def get_workflow(workflow_name="default", sheet_name=None, tab_name=None):
    """Returns a list of namedtuples that define a series of tasks to perform."""
    
    print('Preparing to run workflow: "%s".' % workflow_name)

    from collections import namedtuple
    import sys, os, pickle
    import functions
    import private
    
    # First, we define the language we are going to use in constructing workflows.
    Configure = namedtuple('Configure', 'message repeat_seconds')
    Pipulate  = namedtuple('Pipulate',  'message sheet tab args_tab rows_to_batch chunk_limit extra_data cache')
    Populate  = namedtuple('Populate',  'message sheet tab list_of_rows cols rows_to_batch chunk_limit')
    Append    = namedtuple('Append',    'message sheet tab list_of_rows')

    # Next, we get the Google Sheet and Tab name from user, setting it as the default for future runs.
    
    #if workflow_name in ['default', 'throttled', 'dribble', 'test'] and not sheet_name and not tab_name:
    default_sheet, default_tab = None, None
    if os.path.isfile("defaults.pkl"):
        defaults = pickle.load(open("defaults.pkl", "rb"))
        default_sheet = defaults['sheet_name']
        default_tab = defaults['tab_name']
    if default_sheet:
        sheet_name = input("Google Sheet name (or hit Enter to use %s): " % default_sheet) or default_sheet
    else:
        sheet_name = input("Google Sheet name: ")
    print("Sheet: %s" % sheet_name)
    if default_tab:
        tab_name = input("Google Tab name (or hit Enter to use %s): " % default_tab) or default_tab
    else:
        tab_name = input("Google Tab name: ")
    print("Tab: %s" % tab_name)
    #End if-indent
    
    defaults = {"sheet_name": sheet_name, "tab_name": tab_name}
    pickle.dump(defaults, open("defaults.pkl", "wb"))

    # Thirdly, we set a bunch of local values that will help us write our workflows.
    dates = functions.common_date_boundaries()
    start = dates.start_30_days
    end = dates.end_30_days
    site = private.site  # <-- Site URL for seoinit
    gaid = private.gaid  # <-- Google Analytics ID for seoinit
    feed = private.feed  # <-- Atom Feed for feedmonitor
    feed_sheet = private.feed_sheet
    google_search_console = "functions.populate_from_gsc('%s', '%s', '%s')" % (site, start, end)
    google_analytics = "functions.populate_from_ga('%s','ga:%s', '%s', '%s')" % (site, gaid, start, end)
    one_page_crawl = "functions.one_page_crawl('%s')" % (site)
    atom_feed = "functions.populate_from_atom('%s', 'Feed', '%s')" % (feed_sheet, feed)

    # Lastly, we create a bunch of different workflows we can choose from.
    workflows = {
        "default":[  
            Pipulate(
                message="Workflow: default - Pipulating 5 rows at a time up to 1000.",
                sheet=sheet_name,
                tab=tab_name,
                args_tab="kwargs",
                rows_to_batch=5,
                chunk_limit=200,
                extra_data=None,
                cache=False,
            )
        ],
        "onerow":[  
            Pipulate(
                message="Workflow: onerow - Processing just 1 row then stopping.",
                sheet=sheet_name,
                tab=tab_name,
                args_tab="kwargs",
                rows_to_batch=3,
                chunk_limit=1,
                extra_data=None,
                cache=False,
            )
        ],
        "dribble":[  
            Configure(
                message="Workflow: dribble = Processing 9 rows every 11 minutes.",
                repeat_seconds=660
            ),
            Pipulate(
                message="Dribbling.",
                sheet=sheet_name,
                tab=tab_name,
                args_tab="kwargs",
                rows_to_batch=9,
                chunk_limit=1,
                extra_data=None,
                cache=False,
            )
        ],
        "tests":[  
            Populate(
                message="Establish that we have Internet connectivity and can fully resolve http requests.",
                sheet="Tests",
                tab="Sheet1",
                list_of_rows = 'functions.topsites()',
                cols = None,
                rows_to_batch=50,
                chunk_limit=0
            ),
            Pipulate(
                message="Default: Processing 20 updates of 50 rows at a time (1000 rows total).",
                sheet="Tests",
                tab="Sheet1",
                args_tab=None,
                rows_to_batch=5,
                chunk_limit=5,
                extra_data=None,
                cache=False,
            )
        ],
        "seoinit":[
            Populate(
                message="SEOInit: Caries out SEO site audit, using Google Analytics & Search Console access.",
                sheet=sheet_name,
                tab="GSC",
                list_of_rows = google_search_console,
                cols = None,
                rows_to_batch=50,
                chunk_limit=None
            ),
            Populate(
                message=None,
                sheet=sheet_name,
                tab="GA",
                list_of_rows = google_analytics,
                cols = None,
                rows_to_batch=50,
                chunk_limit=None
            ),
            Populate(
                message=None,
                sheet=sheet_name,
                tab="Crawl",
                list_of_rows = one_page_crawl,
                cols = None,
                rows_to_batch=50,
                chunk_limit=None
            )
        ],
        "gsc":[
            Populate(
                message="GSC: Pulls last 30 days of Google Search Console data.",
                sheet=sheet_name,
                tab="GSC",
                list_of_rows = google_search_console,
                cols = None,
                rows_to_batch=50,
                chunk_limit=None
            )
        ],
        "quickcrawl":[
            Populate(
                message="QuickCrawl: Grabs all onsite links from a page of site (usually homepage).",
                sheet=sheet_name,
                tab="Crawl",
                list_of_rows = one_page_crawl,
                cols = None,
                rows_to_batch=50,
                chunk_limit=None
            )
        ],
        "feedmonitor":[
            Configure(
                message="FeedMonitor: Checks ATOM feed and appends new items to bottom of spreadsheet.",
                repeat_seconds=660
            ),
            Append(
                message="Appending rows to the Feed Monitor sheet.",
                sheet=feed_sheet,
                tab="Feed",
                list_of_rows = atom_feed,
            ),
            Pipulate(
                message="Pipulating the newly added rows.",
                sheet=feed_sheet,
                tab="Feed",
                args_tab="kwargs",
                rows_to_batch=5,
                chunk_limit=200,
                extra_data=None,
                cache=False,
            )
        ]
    }
    workflow = workflows[workflow_name]
    return workflow